In [1]:
import os, boto3, requests, io, json

In [2]:
with open("credentials/aws_keys.json") as f:
    keys = json.load(f)

os.environ["AWS_ACCESS_KEY_ID"] = keys["AWS_ACCESS_KEY_ID"]
os.environ["AWS_SECRET_ACCESS_KEY"] = keys["AWS_SECRET_ACCESS_KEY"]
os.environ["AWS_DEFAULT_REGION"] = keys["AWS_DEFAULT_REGION"]

with open("credentials/ree.json") as f:
    ree = json.load(f)

os.environ["REE_API_KEY"] = ree["REE_API_KEY"]

In [12]:
API_TOKEN = os.environ["REE_API_KEY"]
S3_BUCKET = "hab-ree-data"
S3_PREFIX = "demanda-real"

In [13]:
s3 = boto3.client("s3")

buckets = [b["Name"] for b in s3.list_buckets()["Buckets"]]
print("Buckets:", buckets)

Buckets: ['hab-ree-data']


In [14]:
# Comprobar el indicator_id con el código de ree_indicators. Lo filtré para ver los que contienen "demanda real"

INDICATOR_ID = "1293"
year = "2025"
month = "01"
day = "01"

url = f"https://api.esios.ree.es/indicators/{INDICATOR_ID}"
headers = {
    "x-api-key": API_TOKEN,
    "Accept": "application/json",
    "Content-Type": "application/json"
}
params = {
    "start_date": f"{year}-{month}-{day}T00",
    "end_date": f"{year}-{month}-{day}T23"
}

In [ ]:
# Descargar datos

resp = requests.get(url, headers=headers, params=params)
resp.raise_for_status()
data = resp.json()

#data

In [16]:
# Guardar datos como JSON

body = json.dumps(data["indicator"]["values"], ensure_ascii=False, indent=2)

#body

In [17]:
# Definir la ruta en S3

key = f"{S3_PREFIX}/{year}/{month}/{day}/part-test.json"

In [18]:
# Subir a S3
s3.put_object(Bucket=S3_BUCKET, Key=key, Body=body, ContentType="application/json")

print("Subido a S3:", f"s3://{S3_BUCKET}/{key}")

Subido a S3: s3://hab-ree-data/demanda-real/2025/01/01/part-test.json
